In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,  roc_auc_score
import numpy as np
import time
import pickle
"Own made libraries"
import database as db
import scrapping.scrapperSelenium as scrapperSelenium
import scrapping.scrapperSoup as scrapperSoup
import textCleaning as tc
import model.buildmodel as build

[nltk_data] Downloading package wordnet to C:\Users\Roy
[nltk_data]     Dijkstra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Roy
[nltk_data]     Dijkstra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Roy
[nltk_data]     Dijkstra\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
dfReviewsDb = db.retrieve_table_into_df('cleanedhoteldata')
X = dfReviewsDb["lemReviews"]
y = dfReviewsDb["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=69)

In [9]:
vect = CountVectorizer(max_features=1000)

X_train_vect = vect.fit_transform(X_train)
X_test_vect = vect.transform(X_test)

In [6]:
randomForest = pickle.load(open('model/saved/finalized_random_model_best_params2.sav', 'rb'))

In [14]:
y_pred = randomForest.predict(X_test_vect)
y_prob = randomForest.predict_proba(X_test_vect)[::, 1]

In [15]:
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("AUC: \n", roc_auc_score(y_test, y_prob))

Accuracy: 92.12%
Confusion Matrix:
 [[   661  31751]
 [   742 379453]]
AUC: 
 0.6965041215355987


In [8]:
import pickle
filename = 'finalized_model_best_params.sav'
pickle.dump(randomForest, open(filename, 'wb'))

## Cleaning data from Hotel_Reviews.csv

In [2]:
start_time = time.time()

dfReviews = pd.read_csv("Hotel_Reviews.csv")
dfReviews = tc.kaggle_strip_reviews(dfReviews)
dfReviews = tc.kaggle_label_data(dfReviews)

E:\Roy Dijkstra\School\BigData\BDSE\Individual Assignment 1\textCleaning.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfStrippedReviews["Review"] = dfReviews["Negative_Review"].apply(lambda x: x.replace("No Negative", "")) + \


In [4]:
dfReviews = tc.clean_data(dfReviews)

end_time = time.time()

In [6]:
db.insert_df_into_db(dfReviews, "cleanedhoteldata", "append")

Inserting data into database per 10000 rows


In [7]:
time_lapsed = end_time - start_time
print(time_lapsed)

282.41862750053406


### Clean own data

In [3]:
dfReviewsOwn = pd.read_csv("data/own_reviews.csv", sep=',')
dfReviewsOwn = tc.own_label_data(dfReviewsOwn)
dfReviewsOwn = tc.clean_data(dfReviewsOwn)

#### Insert own data into cleanedhoteldata database

In [ ]:
db.insert_df_into_db(dfReviewsOwn, "cleanedhoteldata", "append")

In [ ]:
dfReviews = db.retrieve_table_into_df('cleanedhoteldata')
svm = build.build_svm(dfReviews)
filename = 'model/svm_best_params.sav'
pickle.dump(randomForest, open(filename, 'wb'))

In [14]:
dfReviewsOwn = db.retrieve_table_into_df('scrappedrawhoteldata')
dfReviewsOwn = tc.clean_data(dfReviewsOwn)

In [15]:
Xown = dfReviewsOwn['lemReviews']
Yown = dfReviewsOwn['label']

Xown_vect = vect.transform(Xown)

In [16]:
y_pred = randomForest.predict(Xown_vect)
y_prob = randomForest.predict_proba(Xown_vect)[::, 1]
print("Accuracy: {:.2f}%".format(accuracy_score(Yown, y_pred) * 100))
print("Confusion Matrix:\n", confusion_matrix(Yown, y_pred))
print("AUC: \n", roc_auc_score(y_test, y_prob))

Accuracy: 94.00%
Confusion Matrix:
 [[  1  12]
 [  0 187]]


ValueError: Found input variables with inconsistent numbers of samples: [103152, 200]